In [101]:
import numpy as np
import random
def iniciar_tablero(n_filas, start,m2):

    for c in m2.valores:
        c.destroy()
    for c in m2.celdas:
        c.destroy()
    del m1.nueva_lista[:]
    
    #LECTURA DE LA MATRIZ DE LA INTERFAZ DE USUARIO, ES LA QUE USARÉ
    n_columnas= int(entry_columna.get())
    lista_tablero=[]
    T=[]    #Array de valores de la matriz que son TRAMPA
    B=[]    #Array de valores de la matriz que son BONIFICACIONES
    for c in m1.celdas:
        try:
            lista_tablero.append(int(c.get()))  #INTENTO PASAR A ENTERO EL VALOR DE LA CASILLA, SI NO ME DEJA ES PORQUE ES UNA CASILLA TRAMPA O BONUS
        except ValueError:
            recompensa = str(c.get())
            if(recompensa.startswith("T")):             #SI EMPIEZA POR T ES TRAMPA -> (T,6)
                T_aux = recompensa.split(",")           #AUX ES LISTA DE DOS ELEMENTOS [[T],[6]]
                T_aux.remove("T")                       #ELIMINO LA T DE LA LISTA
                val=T_aux[0]                            #ME QUEDO CON EL NUMERO
                T.append(int(val))
                lista_tablero.append(int(val))
            else:
                B_aux = recompensa.split(",")
                B_aux.remove("B")
                val=B_aux[0]
                B.append(int(val))            
                lista_tablero.append(int(val))

    set_tablero= set(lista_tablero)

    inic_def= int(entry_inic.get())
    goal_def= int(entry_fin.get())
    iteraciones= int(entry_iteraciones.get())
    gamma_def= float(entry_gamma.get()) 

    #VALIDACION DE FORMULARIO#
    #VALIDACION INICIO Y GOAL DENTRO DE MATRIZ DE NUMEROS
    if(not(goal_def in lista_tablero) or not(inic_def in lista_tablero)):
        mb.showerror("Error","Posición de Inicio y Objetivo han de ser números que existan en la matriz")
    #MATRIZ DE NUMEROS
    if(len(set_tablero) < len(lista_tablero)):
        mb.showerror("Error","No puede repetir numero en la matriz")

    #VALIDACION DE TRAMPA Y BONUS: NO PUEDE HABER TRAMPA NI BONUS EN INICIO NI FIN
    if((goal_def in T) or (goal_def in B) or (inic_def in T) or (inic_def in B)):
        mb.showerror("Error","Posición de Inicio y Objetivo no deben de ser posiciones de bonus ni trampa")

    #VALIDACION GAMMA EPSILON Y ALFA: ENTRE 0 Y 1
    if(not(gamma_def>0 and gamma_def<1)):
        mb.showerror("Error","Gamma debe de estar entre 0 y 1")

    print("T",T)
    print("B",B)
    print("Lista",lista_tablero)
    print("Set",set_tablero)
    
    lista_tablero = [lista_tablero[i:i+n_columnas] for i in range(0, len(lista_tablero), n_columnas)]
    m1.nueva_lista=lista_tablero

    #A CONTINUACIÓN VAMOS A DEFINIR TODAS LAS PROPIEDADES DEL PROBLEMA
    tablero_array = [[0,1],[2,3],[4,5]]
    #tablero_array = [[0,1,2],[3,4,5],[6,7,8]]
    tablero_array = lista_tablero #TABLERO QUE VAMOS A UTILIZAR
    tablero = np.array(tablero_array) #AQUÍ LO CONVERTIMOS EN UNA MATRIZ
    casillas = len(tablero)*len(tablero[0]) #NÚMERO DE CASILLAS DEL TABLERO
    goal = int(goal_def) #META PARA TERMINAR EL ENTRENAMIENTO
    Q = np.zeros((casillas, casillas), dtype = int) #MATRIZ DE ENTRENAMIENTO INICIALIZADA A 0
    R = np.full((casillas, casillas), -1) #MATRIZ DE MOVIMIENTOS INICIALIZADA A -1

    #DECLARACIÓN DE R, LA MATRIZ DE MOVIMIENTOS POSIBLES. UNA POSICIÓN DE ESTA MATRIZ TENDRÁ EL VALOR 0 CUANDO EXISTA CAMINO
    #ENTRE X (STATE) E Y (ACTION). SI UNO DE ESOS CAMINOS VÁLIDOS CONDUCE A LA META, EL VALOR PASARÁ A SER 100.
    #PARA CAMINOS NO VÁLIDOS (POR NO SER CASILLAS VECINAS), EL VALOR SERÁ -1
    #HEMOS CONSIDERADO QUE LA META ES LA ÚNICA CASILLA CON UN CAMINO QUE LA LLEVA DE VUELTA A ELLA MISMA

    for casilla in range(casillas): #VAMOS A ITERAR POR TODAS LAS CASILLAS DEL TABLERO
        ubicacion = np.where(tablero==casilla) #ALMACENAMOS LAS COORDENADAS DE LA CASILLA DE LA ITERACIÓN ACTUAL

        for y in range(casillas):
            ubicacion_proxima=np.where(tablero==y) #ALMACENAMOS LAS COORDENADAS DE OTRA CASILLA DEL TABLERO PARA COMPROBAR SU
                                                   #CERCANÍA CON LA CASILLA DE LA ITERACIÓN ACTUAL
            #COMPROBACIÓN HORIZONTAL Y VERTICAL
            if(ubicacion[0] == ubicacion_proxima[0]+1 and ubicacion[1] == ubicacion_proxima[1]):
                if(y==goal):
                    R[casilla,y] = 100
                elif(y in B):
                    R[casilla,y] = 60
                elif(y in T):
                    R[casilla,y] = -1
                else:
                    R[casilla,y] = 0

            if(ubicacion[0] == ubicacion_proxima[0] and ubicacion[1] == ubicacion_proxima[1]+1):
                if(y==goal):
                    R[casilla,y] = 100
                elif(y in B):
                    R[casilla,y] = 60
                elif(y in T):
                    R[casilla,y] = -1
                else:
                    R[casilla,y] = 0

            if(ubicacion[0] == ubicacion_proxima[0]-1 and ubicacion[1] == ubicacion_proxima[1]):
                if(y==goal):
                    R[casilla,y] = 100
                elif(y in B):
                    R[casilla,y] = 60
                elif(y in T):
                    R[casilla,y] = -1
                else:
                    R[casilla,y] = 0

            if(ubicacion[0] == ubicacion_proxima[0] and ubicacion[1] == ubicacion_proxima[1]-1):
                if(y==goal):
                    R[casilla,y] = 100
                elif(y in B):
                    R[casilla,y] = 60
                elif(y in T):
                    R[casilla,y] = -1
                else:
                    R[casilla,y] = 0


            #COMPROBACIÓN DIAGONAL
            if(ubicacion[0] == ubicacion_proxima[0]+1 and ubicacion[1] == ubicacion_proxima[1]+1):
                if(y==goal):
                    R[casilla,y] = 100
                elif(y in B):
                    R[casilla,y] = 60
                elif(y in T):
                    R[casilla,y] = -1
                else:
                    R[casilla,y] = 0

            if(ubicacion[0] == ubicacion_proxima[0]-1 and ubicacion[1] == ubicacion_proxima[1]-1):
                if(y==goal):
                    R[casilla,y] = 100
                elif(y in B):
                    R[casilla,y] = 60
                elif(y in T):
                    R[casilla,y] = -1
                else:
                    R[casilla,y] = 0

            if(ubicacion[0] == ubicacion_proxima[0]-1 and ubicacion[1] == ubicacion_proxima[1]+1):
                if(y==goal):
                    R[casilla,y] = 100
                elif(y in B):
                    R[casilla,y] = 60
                elif(y in T):
                    R[casilla,y] = -1
                else:
                    R[casilla,y] = 0

            if(ubicacion[0] == ubicacion_proxima[0]+1 and ubicacion[1] == ubicacion_proxima[1]-1):
                if(y==goal):
                    R[casilla,y] = 100
                elif(y in B):
                    R[casilla,y] = 60
                elif(y in T):
                    R[casilla,y] = -1
                else:
                    R[casilla,y] = 0

            #NO HAY ACTIONS HACIA UNO MISMO, EXCEPTO QUE SEAS LA META
            if(casilla==y):
                if(y==goal):
                    R[casilla,y] = 100
                else:
                    R[casilla,y] = -1

    
    m2.celdas=[]
    m2.valores=[]
    text_q=Label(ventana, text="Matriz Q")
    text_q.grid(column=0,row=20)
    
    salida_Q=Label(ventana, text=Q)
    salida_Q.grid(column=0,row=21)
    m2.celdas.append(text_q)
    m2.valores.append(salida_Q)

    text_r=Label(ventana, text="Matriz R")
    text_r.grid(column=1, row=20)
    salida_R = Label(ventana,text=R)
    salida_R.grid(column=1, row=21)
    m2.celdas.append(text_r)
    m2.celdas.append(salida_R)

    print("Matriz:\n", tablero)
    print("Número de casillas: ", casillas)
    print("\nR (matriz de movimientos):\n", R)
    print("\nQ (matriz de aprendizaje):\n", Q)
    print("\n")
    start_program = Button(ventana, text="Entrenar matriz", command=lambda: entrenamiento(inic_def, n_filas, start, Q, R,m2))
    start_program.grid(column=0, row=n_filas+11)
    start_program_alternative = Button(ventana, text="Entrenamiento alternativo", command=lambda: entrenamiento_alternativo(inic_def, n_filas, start, Q, R,m2))
    start_program_alternative.grid(column=1, row=n_filas+11)
    start.append(start_program_alternative) #LLAMADA A ENTRENAR


In [102]:
import matplotlib.pyplot as plt
import matplotlib 
matplotlib.use("TkAgg") 
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,NavigationToolbar2Tk)
from numpy import *
import networkx as nx

def entrenamiento(inic_def, n_filas, start, Q, R, m2):
    
    for c in m2.valores:
        c.destroy()

    goal= int(entry_fin.get()) #META DEL RECORRIDO
    episodios= int(entry_iteraciones.get())#NÚMERO DE ITERACIONES QUE TENDRÁ EL ENTRENAMIENTO
    gamma= float(entry_gamma.get()) #FACTOR DE APRENDIZAJE
    state = random.randint(0,8) #ESTADO ALEATORIO DE INICIO DEL ENTRENAMIENTO 
    print("Estado inicial escogido: ", state)
    print("Meta escogida: ", goal)
    print("Gamma: ", gamma)
    print("Episodios: ", episodios)
    
    if(len(datos.entrenamientos)==0):
        cont= 1
    else:
        cont=datos.entrenamientos[-1]+1

    edges=[]    #Aristas del grafo
    edges_choice=[] #Aristas de las acciones escogidas

    for x in range(episodios):
        print("Estado actual: ", state)
        actions = np.where(R[state]!=-1)[0] #LISTA DE ACCIONES QUE SON POSIBLES
        print("Posibles acciones: ", actions)
        random_action = random.choice(actions)
        print("Acción escogida al azar: ", random_action) #ESCOJO UNA ACCIÓN ALEATORIAMENTE, LA CUAL SERÁ MI PRÓXIMO ESTADO
        print("Peso en R de esa acción: ", R[state, random_action])
        next_actions = np.where(R[random_action]!=-1)[0] #LISTA DE ACCIONES QUE SON POSIBLES EN EL PRÓXIMO ESTADO
        print("Acciones que serán posibles cuando esté en", random_action, ": ", next_actions)

        for a in actions:
            edges.append((state,a))

        edges_choice.append((state,random_action))    #COLOR ROJO

        next_actions_Qvalue = np.ones(len(next_actions), dtype=int)

        for y in range(len(next_actions)):
            next_actions_Qvalue[y] = Q[random_action, next_actions[y]]

        print("Peso en Q para esas futuras acciones: ", next_actions_Qvalue)

        Q[state, random_action] = R[state, random_action] + gamma * max(next_actions_Qvalue)
        state=random_action
        print("El rendimiento de este episodio de entrenamiento ha sido: ", (Q.sum()/np.max(Q))*100)
        print("\n")

        datos.rendimientos = np.append(datos.rendimientos,(Q.sum()/np.max(Q))*100)
        datos.entrenamientos= np.append(datos.entrenamientos,cont)
        cont=cont+1

        print(edges)
        if(state==goal):
            print("Hemos alcanzado la meta antes de terminar las iteraciones.")
            break

    G = nx.DiGraph()
    G.add_edges_from(edges)
    black_edges = [edge for edge in G.edges() if edge not in edges_choice]
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 500)
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_edges(G, pos, edgelist=edges_choice, edge_color='r', arrows=True)
    nx.draw_networkx_edges(G, pos, edgelist=black_edges, arrows=False)
    plt.show()
    #Cambiar los NaN obtenidos de rendimiento por 0
    where_are_NaNs = isnan(datos.rendimientos)
    datos.rendimientos[where_are_NaNs] = 0

    print("Entrenamientos",datos.entrenamientos)
    print("Rendimientos",datos.rendimientos)
    graph = Tk()
    graph.title("Rendimientos")
    graph.geometry('800x800')
    f = Figure(figsize=(5,5),dpi=100)
    a = f.add_subplot(111)
    a.plot(datos.entrenamientos, datos.rendimientos)
    a.set_ylabel('Rendimiento')
    a.set_xlabel('Episodio de entrenamiento')

    canvas = FigureCanvasTkAgg(f,master=graph)
    canvas.draw()
    canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)

    toolbar = NavigationToolbar2Tk(canvas, graph)
    toolbar.update()
    canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)

    salida_Q=Label(ventana, text=Q)
    salida_Q.grid(column=0,row=21)
    m2.valores.append(salida_Q)
    print(Q)
    
    finish_program = Button(ventana, text="Recorrer tablero", command=lambda: recorrer_tablero(goal, Q, R))
    finish_program.grid(column=0, row=n_filas+21)
    start.append(finish_program) #LLAMADA A RECORRER EL TABLERO

In [103]:
def entrenamiento_alternativo(inic_def, n_filas, start, Q, R, m2):
    
    for c in m2.valores:
        c.destroy()

    #inic_def= int(entry_inic.get())
    goal= int(entry_fin.get()) #META DEL RECORRIDO
    episodios= int(entry_iteraciones.get())#NÚMERO DE ITERACIONES QUE TENDRÁ EL ENTRENAMIENTO
    gamma= float(entry_gamma.get()) #FACTOR DE APRENDIZAJE
    state = random.randint(0,len(Q[0])-1) #ESTADO ALEATORIO DE INICIO DEL ENTRENAMIENTO 
    
    epsilon = float(entry_epsilon.get()) #HAY QUE METERLO POR INTERFAZ DE USUARIO
    alpha =  float(entry_alpha.get())  #HAY QUE METERLO POR INTERFAZ DE USUARIO
    
    #VALIDACION DE EPSILON Y ALPHA


    print("Estado inicial escogido: ", state)
    print("Meta escogida: ", goal)
    print("Gamma: ", gamma)
    print("Epsilon: ", epsilon)
    print("Alpha: ", alpha)
    print("Episodios: ", episodios)

    if(len(datos.entrenamientos)==0):
        cont= 1
    else:
        cont=datos.entrenamientos[-1]+1

    edges=[]    #Aristas del grafo
    edges_choice=[] #Aristas de las acciones escogidas

    for x in range(episodios):
        print("Estado actual: ", state)
        actions = np.where(R[state]!=-1)[0] #LISTA DE ACCIONES QUE SON POSIBLES
        print("Posibles acciones: ", actions)
        
        for a in actions:
            edges.append((state,a))

        if(random.uniform(0, 1) < epsilon):
            print("Se ha ejecutado la Fase 1.")
            next_state = random.choice(actions) #ESCOJO UNA ACCIÓN ALEATORIAMENTE, LA CUAL SERÁ MI PRÓXIMO ESTADO
            print("Acción escogida al azar: ", next_state)
        else:
            print("Se ha ejecutado la Fase 2.")
            next_state = 0
            Q_aux = -1
            
            for st in actions:                  #EN ESTE BUCLE NOS QUEDAREMOS CON EL PRÓXIMO ESTADO CON MAYOR VALOR EN Q
                if(Q[state, st] > Q_aux):            
                    Q_aux = Q[state, st]
                    next_state = st

            print("Acción escogida con el máximo Q: ", next_state)

        edges_choice.append((state,next_state))    #COLOR ROJO DEL GRAFO, ACCIONES TOMADAS

        print("Peso en R de esa acción: ", R[state, next_state])
        next_actions = np.where(R[next_state]!=-1)[0] #LISTA DE ACCIONES QUE SON POSIBLES EN EL PRÓXIMO ESTADO
        print("Acciones que serán posibles cuando esté en", next_state, ": ", next_actions)

        next_actions_Qvalue = np.ones(len(next_actions), dtype=int)

        for y in range(len(next_actions)):            #MEDIANTE ESTE BUCLE, DEJAREMOS PREPARADO UN ARRAY CON LOS VALORES EN Q 
            next_actions_Qvalue[y] = Q[next_state, next_actions[y]]        #DEL FUTURO ESTADO, ASÍ PODREMOS SACARLE EL MAX

        print("Peso en Q para esas futuras acciones: ", next_actions_Qvalue)

        Q[state, next_state] = R[state, next_state] + gamma * max(next_actions_Qvalue)
        state = next_state
        epsilon = epsilon * alpha
        
        print("El rendimiento de este episodio de entrenamiento ha sido: ", (Q.sum()/np.max(Q))*100) #RENDIMIENTO
        print("\n")

        datos.rendimientos = np.append(datos.rendimientos,(Q.sum()/np.max(Q))*100)
        datos.entrenamientos= np.append(datos.entrenamientos,cont)
        cont=cont+1

        if(state==goal):
            print("Hemos alcanzado la meta antes de terminar las iteraciones.")
            break

    G = nx.DiGraph()
    G.add_edges_from(edges)
    black_edges = [edge for edge in G.edges() if edge not in edges_choice]
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 500)
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_edges(G, pos, edgelist=edges_choice, edge_color='r', arrows=True)
    nx.draw_networkx_edges(G, pos, edgelist=black_edges, arrows=False)
    plt.show()

    #Cambiar los NaN obtenidos de rendimiento por 0
    where_are_NaNs = isnan(datos.rendimientos)
    datos.rendimientos[where_are_NaNs] = 0

    print("Entrenamientos",datos.entrenamientos)
    print("Rendimientos etrenamiento",datos.rendimientos)

    graph = Tk()
    graph.title("Rendimientos Alternativos")
    graph.geometry('800x800')
    f = Figure(figsize=(5,5),dpi=100)
    a = f.add_subplot(111)
    a.plot(datos.entrenamientos, datos.rendimientos)
    a.set_ylabel('Rendimiento')
    a.set_xlabel('Episodio de entrenamiento')

    canvas = FigureCanvasTkAgg(f,master=graph)
    canvas.draw()
    canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)

    toolbar = NavigationToolbar2Tk(canvas, graph)
    toolbar.update()
    canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)
    
    entry_epsilon.delete(0,END)
    entry_epsilon.insert(END,epsilon) #ACTUALIZAMOS EL VALOR DE EPSILON EN LA INTERFAZ DE USUARIO
    
    salida_Q=Label(ventana, text=Q)
    salida_Q.grid(column=0,row=21)
    m2.valores.append(salida_Q)
    print(Q)
    
    finish_program = Button(ventana, text="Recorrer tablero", command=lambda: recorrer_tablero(goal, Q, R))
    finish_program.grid(column=0, row=n_filas+21)
    start.append(finish_program) #LLAMADA A RECORRER EL TABLERO

In [104]:
def recorrer_tablero(goal_def, Q, R): 

    for c in recorrido.label:
        c.destroy()
        
    state = int(entry_inic.get()) #ESTADO DE INICIO DEL RECORRIDO
    goal = int(goal_def) #META DEL RECORRIDO
    contador = 0 #NÚMERO DE MOVIMIENTOS USADOS PARA LLEGAR A LA META

    print("El estado incial es: ", state)
    print("La meta es: ", goal)

    edges=[]    #Aristas del grafo
    edges_choice=[] #Aristas de las acciones escogidas
    
    #VAMOS A HACER QUE EL AGENTE RECORRA EL TABLERO Y RECORRA EL MEJOR CAMINO
    while state!=goal:
        print("\nEstado actual: ", state)
        actions = np.where(R[state]!=-1)[0] #LISTA DE ACCIONES QUE SON POSIBLES EN ESTE MOMENTO
        print("Posibles acciones: ", actions)

        for a in actions:                   #AÑADIR VERTICES DE POSIBLES ACCIONES
            edges.append((state,a))

        values = np.ones(len(actions), dtype=int) #PESO QUE TIENEN ESAS ACCIONES EN LA MATRIZ Q            
        for y in range(len(actions)):
            values[y] = Q[state, actions[y]]   
        print("Pesos de esas acciones: ", values) 

        posicion_mayor_peso = np.where(values==np.max(values))[0][0] #LA POSICIÓN EN EL ARRAY DONDE ESTÁ LA ACCIÓN DE MAYOR PESO
        print("Posición del mejor movimiento, en base a su peso: ", posicion_mayor_peso)
        best_action = actions[posicion_mayor_peso] #LA MEJOR ACCIÓN POSIBLE
        print("Mejor acción: ", best_action)

        edges_choice.append((state,best_action))    #COLOR ROJO DEL GRAFO, ACCIONES TOMADAS

        state=best_action
        contador = contador + 1
        
        if(contador==20):
            print("El agente no está lo suficientemente entrenado y no encuentra la meta, finalizando recorrido.")
            mb.showerror("Error","El agente no está lo suficientemente entrenado y no encuentra la meta, finalizando recorrido.")
            break

    if(contador<20):
        print("Hicieron falta", contador, "movimientos para llegar a la meta.\n")
        res = Label(ventana,text="Hicieron falta " + str(contador) + " movimientos para llegar a la meta.")
        res.grid(column=0,row=60)
        recorrido.label.append(res)

        G = nx.DiGraph()
        G.add_edges_from(edges)
        black_edges = [edge for edge in G.edges() if edge not in edges_choice]
        pos = nx.spring_layout(G)
        nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 500)
        nx.draw_networkx_labels(G, pos)
        nx.draw_networkx_edges(G, pos, edgelist=edges_choice, edge_color='r', arrows=True)
        nx.draw_networkx_edges(G, pos, edgelist=black_edges, arrows=False)
        plt.show()

In [105]:
from tkinter import *
from tkinter import messagebox as mb 
from random import sample   
import numpy as np
#Iniciamos la ventana Principal
ventana = Tk()
ventana.title("Aprendizaje por Refuerzo")
ventana.geometry('1200x1200')
#ventana.configure(background = 'dark turquoise')

#Añadimos las entradas de texto de Nº de Filas y Columnas
filas = Label(ventana,text="Numero de Filas")
entry_fila = Entry(ventana, width=5)
filas.grid(column=0,row=0)
entry_fila.grid(column=1,row=0)

columnas = Label(ventana,text="Numero de Columnas")
entry_columna = Entry(ventana, width=5)
columnas.grid(column=0,row=3)
entry_columna.grid(column=1,row=3)

inic = Label(ventana,text="Posicion de inicio")
entry_inic = Entry(ventana, width=5)
entry_inic.insert(END,0)
inic.grid(column=2,row=0)
entry_inic.grid(column=3,row=0)

fin = Label(ventana,text="Objetivo")
entry_fin = Entry(ventana, width=5)
entry_fin.insert(END,6)
fin.grid(column=2,row=3)
entry_fin.grid(column=3,row=3)

gamma = Label(ventana,text="Valor Gamma")
entry_gamma = Entry(ventana, width=5)
entry_gamma.insert(END,0.5)
gamma.grid(column=4,row=0)
entry_gamma.grid(column=5,row=0)

iteraciones = Label(ventana,text="Numero de Iteraciones")
entry_iteraciones = Entry(ventana, width=5)
entry_iteraciones.insert(END,3)
iteraciones.grid(column=4,row=3)
entry_iteraciones.grid(column=5,row=3)

epsilon = Label(ventana,text="Valor Epsilon")
entry_epsilon = Entry(ventana, width=10)
entry_epsilon.insert(END,0.8)
epsilon.grid(column=6,row=0)
entry_epsilon.grid(column=7,row=0)

alpha = Label(ventana,text="Valor Alpha")
entry_alpha = Entry(ventana, width=5)
entry_alpha.insert(END,0.9)
alpha.grid(column=6,row=3)
entry_alpha.grid(column=7,row=3)


#Clase Matriz
class Matrix():
    def __init__(self,celdas,pos,valores,start,nueva_lista):
        self.celdas=celdas[:]
        self.pos=pos[:]
        self.valores=valores[:]
        self.start=start[:]
        self.nueva_lista=nueva_lista[:]

#Clase Recorrido (muestra datos de recorrido final)
class Recorrido():
    def __init__(self,label):
        self.label=label[:]
        
recorrido = Recorrido([])

#Clase EntrenamientoAlternativo
class Alternative():
    def __init__(self,label,epsilon,alfa):
        self.label=label[:]
        self.epsilon=epsilon[:]
        self.alfa=alfa[:]

alternative = Alternative([],[],[])

#Clase datos (entrenamientos y rendimientos)
class Datos:
    def __init__(self,entrenamientos, rendimientos):
        self.entrenamientos=entrenamientos[:]
        self.rendimientos=rendimientos[:]
datos = Datos([],[])

#m1 instanciacion de matriz vacía
m1 = Matrix([],[],[],[],[])
m2 = Matrix([],[],[],[],[])


def aleatorizar(e,n,valores):
    
    random = sample([x for x in range(0,n)],n)[0]
    while(random in valores):
        random = sample([x for x in range(0,n)],n)[0]
    valores.append(random)
    e.insert(END,random)


def generarMatriz(m1):

#En el caso de que m1 sea una matriz ya iniciada, esta tendrá celdas, valores, y posiciones que hay que eliminar previamente    
    celdas = m1.celdas
    pos = m1.pos
    valores = m1.valores
    nueva_lista=m1.nueva_lista
    start=m1.start

    del m1.pos[:]
    del m1.valores[:]
    for b in start:
        b.destroy()
    del m1.nueva_lista[:]
    for c in celdas:
        c.destroy()

#Una vez eliminados validamos las entradas de nfilas y ncolumnas
    validate_tam(entry_fila.get(),entry_columna.get())
    n_filas=int(entry_fila.get())
    n_columnas= int(entry_columna.get())
    total = n_filas * n_columnas

    # Creamos la tabla de entradas
    for row in range(n_filas):
        for column in range(n_columnas):
            index = (row, column)
            e = Entry(ventana, width=10)
            aleatorizar(e,total,valores)
            e.grid(row=row+10, column=column, stick="nsew")
            celdas.append(e)
            pos.append(index)

            
    m1.celdas= celdas
    m1.pos = pos
    m1.valores= valores
    result = zip(pos,valores)
    result_set = set(result)
    inic_QR = Button(ventana, text= "Generar Q y R", command=lambda: 
    iniciar_tablero(n_filas, start,m2))
    inic_QR.grid(column=2, row=4)
    #AQUÍ SE HACE LA MAGIA

def validate_tam(n_filas,n_columnas):
    try:
        f= int(n_filas)
        c= int(n_columnas)
    except ValueError:
        mb.showerror("Error","Debe introducir un valor entero como numero de filas y columnas")
    
    if(int(n_filas) == 0 or (n_columnas) == 0):
        mb.showerror("Error","Ni la fila ni la columna puede ser de tamaño 0")
    
#Creamos la matriz según las entradas
btn = Button(ventana, text="Generar Matriz", command=lambda: generarMatriz(m1))
btn.grid(column=0, row=4)




ventana.mainloop()

sibles acciones:  [0 2 8]
Pesos de esas acciones:  [0 0 0]
Posición del mejor movimiento, en base a su peso:  0
Mejor acción:  0

Estado actual:  0
Posibles acciones:  [1 2 5 7 8]
Pesos de esas acciones:  [0 0 0 0 0]
Posición del mejor movimiento, en base a su peso:  0
Mejor acción:  1

Estado actual:  1
Posibles acciones:  [0 2 8]
Pesos de esas acciones:  [0 0 0]
Posición del mejor movimiento, en base a su peso:  0
Mejor acción:  0

Estado actual:  0
Posibles acciones:  [1 2 5 7 8]
Pesos de esas acciones:  [0 0 0 0 0]
Posición del mejor movimiento, en base a su peso:  0
Mejor acción:  1

Estado actual:  1
Posibles acciones:  [0 2 8]
Pesos de esas acciones:  [0 0 0]
Posición del mejor movimiento, en base a su peso:  0
Mejor acción:  0

Estado actual:  0
Posibles acciones:  [1 2 5 7 8]
Pesos de esas acciones:  [0 0 0 0 0]
Posición del mejor movimiento, en base a su peso:  0
Mejor acción:  1

Estado actual:  1
Posibles acciones:  [0 2 8]
Pesos de esas acciones:  [0 0 0]
Posición del mejo